In [28]:
import folium
from numpy import mean
from folium.plugins import MarkerCluster
from instagrapi import Client
from random import randint, seed
from time import sleep
from datetime import datetime

In [2]:
# Create client and logining
SESSION_ID = "...." # session_id for loging
cl = Client()
cl.login_by_sessionid(SESSION_ID)

True

In [3]:
USER_NAME= "...."         # User name to analyse
media_cnt = 100           # quantity of media to analyse 
user_id = cl.user_id_from_username(USER_NAME)
media_list = cl.user_medias(user_id, media_cnt)

In [4]:
def get_data_from_media(media) -> dict:
    """
    Function to extract necessary data from media
    """
    media_info = cl.media_info(media.pk).dict()
    if media_info['location']:
        out = {"location_lng": media_info['location']['lng'],
               "location_lat": media_info['location']['lat'],
               "location_name": media_info['location']['name'],
               "like_cnt": media_info['like_count'],
               "desc": media_info['caption_text'],
               "taken_at": media_info['taken_at'],
               "media_type": media_info['media_type']
              }
    else:
        out = {}
    return out

In [5]:
# Collect all data together
data = list()
seed # for randoms
for media in media_list:
    data.append(get_data_from_media(media))
    sleep(randint(10, 200)/100) # to avoid blocking an account

# Filter data without GEO-tags
places = list(filter(lambda item: item.get('location_lat') != None and item.get('location_lng') != None,
                      data
                      )
              )

In [6]:
max_likes = max([item['like_cnt'] for item in places])
min_likes = min([item['like_cnt'] for item in places])

In [8]:
def get_color(n: int) -> str:
    """
    Function to get color from likes's quantity
    """
    borders =[min_likes + i/100*(max_likes-min_likes) for i in range (20, 100, 20)]
    if n > borders[2]:
        return 'darkred' 
    elif n > borders[2]:
        return 'red'    
    elif n > borders[0]:
        return 'lightred'
    else:
        return 'orange'

In [31]:
lat0 = mean([place['location_lat'] for place in places])
lng0 = mean([place['location_lng'] for place in places])

In [34]:
# build map
map = folium.Map(location=[lat0, lng0], zoom_start = 3, control_scale=True)
marker_cluster = MarkerCluster().add_to(map)

for place in places:
    folium.Marker(location=[place['location_lat'], place['location_lng']], 
                  popup=place['taken_at'].strftime('%d-%m-%Y'),# + '\n' + place['desc'], 
                  icon=folium.Icon(color = get_color(place['like_cnt'])))\
    .add_to(marker_cluster)

map.save("map.html")